## Notebook for spike sorting from .kwd data using the Pipeline of mountainsort (linux channel has pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from importlib import reload

# mountainlab imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc
from intan2kwik import kwd

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)
logger.addHandler(ch)
        
logger.info('Logger set')

/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zinch/conda/envs/msort/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
2018-12-18 15:22:20,945 - root - INFO - Logger set
2018-12-18 15:22:20,947 - matplotlib.backends - DEBUG - backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
os.environ['OMP_NUM_THREADS'] = '8'
os.environ['MKL_NUM_THREADS'] = '8'
os.environ['NUMEXPR_NUM_THREADS'] = '8'

### Session parameters and raw files

In [3]:
exp_base = os.path.abspath('/home/zinch/data/')
exp_mnt = os.path.abspath('/mnt/zuperfinchjr/Data/')

file_sys = {'exp_base': exp_base}

sess_par = {'bird': 'o9j9',
           'sess': '20181125_r021_saveontrigger',
            'neural': 'Neural'}


def file_names(sess_par, file_sys=None):
    f_s = unroll_file_sys(file_sys_defaults, file_sys)    
    
    f_n = {}
    f_n['raw'] = os.path.join(f_s['raw'], bird)

In [4]:
raw_folder = os.path.join(exp_mnt, sess_par['bird'], sess_par['neural'], sess_par['sess'])
ss_folder = os.path.join(exp_base, sess_par['bird'], 'ss_data', sess_par['sess'])

logger.info('Raw folder {}'.format(raw_folder))
#glob.glob(os.path.join(raw_folder, '*.rhd'))

2018-12-18 15:22:20,991 - root - INFO - Raw folder /mnt/zuperfinchjr/Data/o9j9/Neural/20181125_r021_saveontrigger


In [5]:
## Convert the whole session to a kwd file
kwd_path = os.path.join(ss_folder, 'experiment.kwd')
os.makedirs(ss_folder, exist_ok=True)
#first_intan_hdr, sess_pd = kwd.intan_to_kwd(raw_folder, kwd_path)

In [6]:
from pipefinch.h5tools.kwik import kwdfunctions as kwdf
from pipefinch.h5tools.kwik import kutil

## revisit reading a dataset

In [28]:
kwd_file = h5py.File(kwd_path, 'r')
dataset = kwd_file['/recordings/0/data']
print(dataset)

<HDF5 dataset "data": shape (14556240, 14), type "<i2">


In [33]:
from pipefinch.h5tools.core import tables

In [50]:
b = dataset[:]

In [57]:
b.size

203787360

In [ ]:
all_dset = dataset.read_direct

In [67]:
# passing stuff to binary
reload(tables)

with open(bin_path, 'wb') as bf:
    stored = tables.dset_to_binary_file(dataset, bf, chan_list=[0])

2018-12-18 16:51:00,622 - pipefinch.h5tools.core.tables - INFO - Ripping dataset from /recordings/0
2018-12-18 16:51:00,623 - pipefinch.h5tools.core.tables - DEBUG - Channel list: [0]
2018-12-18 16:51:00,623 - pipefinch.h5tools.core.tables - DEBUG - About to store 1 entire chunks
2018-12-18 16:51:00,624 - pipefinch.h5tools.core.tables - DEBUG - Chunk start: 0
2018-12-18 16:51:00,624 - pipefinch.h5tools.core.tables - DEBUG -  end: 8000000
2018-12-18 16:51:00,697 - pipefinch.h5tools.core.tables - DEBUG - raw table slice shape (8000000, 1)
2018-12-18 16:51:00,805 - pipefinch.h5tools.core.tables - DEBUG - return table slice shape (8000000, 1)
2018-12-18 16:51:00,824 - pipefinch.h5tools.core.tables - DEBUG - Chunk start: 8000000
2018-12-18 16:51:00,825 - pipefinch.h5tools.core.tables - DEBUG -  end: 14556240
2018-12-18 16:51:00,849 - pipefinch.h5tools.core.tables - DEBUG - raw table slice shape (6556240, 1)
2018-12-18 16:51:00,926 - pipefinch.h5tools.core.tables - DEBUG - return table slice

In [45]:
8000000*14

112000000

In [56]:
407574720/2

203787360.0

In [96]:
## Make .mda file with a set of recordings in a session
reload(kwdf)
reload(kutil)
ch.setLevel(logging.DEBUG)

all_rec_list = kutil.get_rec_list(kwd_path)
all_rec_list

chan_list = np.arange(5)

bin_path = os.path.join(ss_folder, 'experiment.bin')
#bin_file = kwdf.kwd_to_binary(kwd_path, bin_path, chan_list=list(chan_list), rec_list=[0, 5, 10])
rec_attr_dict = kwdf.get_rec_attrs(kwd_path, 0)
all_rec_meta = kwdf.get_all_rec_meta(kwd_path)

In [ ]:
chan_lists = []

In [98]:
all_rec_meta[0]

{'rec_group': {'bit_depth': 16,
  'sample_rate': 20000.0,
  'name': 0,
  'start_sample': 0,
  'start_time': 1},
 'app_data': {'is_multiSampleRate_data': 0,
  'channels_sample_rate': array([20000., 20000., 20000., 20000., 20000., 20000., 20000., 20000.,
         20000., 20000., 20000., 20000., 20000., 20000.]),
  'channel_bit_volts': array([1.9500e-01, 1.9500e-01, 1.9500e-01, 1.9500e-01, 1.9500e-01,
         1.9500e-01, 1.9500e-01, 1.9500e-01, 1.9500e-01, 1.9500e-01,
         1.9500e-01, 1.9500e-01, 1.9500e-01, 5.0354e-05]),
  'channel_names': array([b'D-004', b'D-006', b'D-008', b'D-012', b'D-019', b'D-022',
         b'D-023', b'D-024', b'D-025', b'D-026', b'D-027', b'D-028',
         b'D-029', b'ADC-00'], dtype='|S6')},
 'cont_data': {'valid_samples': array([14556240., 14556240., 14556240., 14556240., 14556240., 14556240.,
         14556240., 14556240., 14556240., 14556240., 14556240., 14556240.,
         14556240., 14556240.])}}

In [154]:
apd = pd.concat([pd.DataFrame([r[k] for r in all_rec_meta]) for k in all_rec_meta[0].keys()], axis=1)
apd.head()

,bit_depth,name,sample_rate,start_sample,start_time,channel_bit_volts,channel_names,channels_sample_rate,is_multiSampleRate_data,valid_samples
0,16,0,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[14556240.0, 14556240.0, 14556240.0, 14556240...."
1,16,1,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[106560.0, 106560.0, 106560.0, 106560.0, 10656..."
2,16,2,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1..."
3,16,3,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[308160.0, 308160.0, 308160.0, 308160.0, 30816..."
4,16,4,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1..."


In [158]:
apd['channel_names'].diff()

TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('S6') dtype('S6') dtype('S6')

In [153]:
pd.concat(pdlist, axis=1)

,bit_depth,name,sample_rate,start_sample,start_time,channel_bit_volts,channel_names,channels_sample_rate,is_multiSampleRate_data,valid_samples
0,16,0,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[14556240.0, 14556240.0, 14556240.0, 14556240...."
1,16,1,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[106560.0, 106560.0, 106560.0, 106560.0, 10656..."
2,16,2,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1..."
3,16,3,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[308160.0, 308160.0, 308160.0, 308160.0, 30816..."
4,16,4,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[1199520.0, 1199520.0, 1199520.0, 1199520.0, 1..."
5,16,5,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[207360.0, 207360.0, 207360.0, 207360.0, 20736..."
6,16,6,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[1164240.0, 1164240.0, 1164240.0, 1164240.0, 1..."
7,16,7,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[774720.0, 774720.0, 774720.0, 774720.0, 77472..."
8,16,8,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[1062720.0, 1062720.0, 1062720.0, 1062720.0, 1..."
9,16,9,20000.0,0,1,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[b'D-004', b'D-006', b'D-008', b'D-012', b'D-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[736560.0, 736560.0, 736560.0, 736560.0, 73656..."


In [132]:
list(all_rec_meta[0].values())

[{'bit_depth': 16,
  'sample_rate': 20000.0,
  'name': 0,
  'start_sample': 0,
  'start_time': 1},
 {'is_multiSampleRate_data': 0,
  'channels_sample_rate': array([20000., 20000., 20000., 20000., 20000., 20000., 20000., 20000.,
         20000., 20000., 20000., 20000., 20000., 20000.]),
  'channel_bit_volts': array([1.9500e-01, 1.9500e-01, 1.9500e-01, 1.9500e-01, 1.9500e-01,
         1.9500e-01, 1.9500e-01, 1.9500e-01, 1.9500e-01, 1.9500e-01,
         1.9500e-01, 1.9500e-01, 1.9500e-01, 5.0354e-05]),
  'channel_names': array([b'D-004', b'D-006', b'D-008', b'D-012', b'D-019', b'D-022',
         b'D-023', b'D-024', b'D-025', b'D-026', b'D-027', b'D-028',
         b'D-029', b'ADC-00'], dtype='|S6')},
 {'valid_samples': array([14556240., 14556240., 14556240., 14556240., 14556240., 14556240.,
         14556240., 14556240., 14556240., 14556240., 14556240., 14556240.,
         14556240., 14556240.])}]

In [133]:
dc = {r[k] for r in all_rec_meta for k in keys}
dc['rec_group']

TypeError: unhashable type: 'dict'

In [ ]:
for k in all_rec_meta[0].keys()

In [134]:
apd.head()

,bit_depth,channel_bit_volts,channel_names,channels_sample_rate,is_multiSampleRate_data,name,sample_rate,start_sample,start_time,valid_samples
0,16.0,NaN,NaN,NaN,NaN,0.0,20000.0,0.0,1.0,NaN
1,16.0,NaN,NaN,NaN,NaN,1.0,20000.0,0.0,1.0,NaN
2,16.0,NaN,NaN,NaN,NaN,2.0,20000.0,0.0,1.0,NaN
3,16.0,NaN,NaN,NaN,NaN,3.0,20000.0,0.0,1.0,NaN
4,16.0,NaN,NaN,NaN,NaN,4.0,20000.0,0.0,1.0,NaN


In [135]:
rec_attr_dict['app_data']['channel_names'][:10]

array([b'D-004', b'D-006', b'D-008', b'D-012', b'D-019', b'D-022',
       b'D-023', b'D-024', b'D-025', b'D-026'], dtype='|S6')

In [136]:
att = kutil.get_rec_attrs(kwd_file, 0)
att

<Attributes of HDF5 object at 139814774525624>

In [137]:
att['start_time']

1

In [138]:
list(att.items())

[('bit_depth', 16),
 ('sample_rate', 20000.0),
 ('name', 0),
 ('start_sample', 0),
 ('start_time', 1)]

In [139]:
from pipefinch.h5tools.core import h5tools

h5tools.obj_attrs_2_dict_translator(kwd_file['/recordings/0/data'])

{'valid_samples': array([14556240., 14556240., 14556240., 14556240., 14556240., 14556240.,
        14556240., 14556240., 14556240., 14556240., 14556240., 14556240.,
        14556240., 14556240.])}

In [140]:
def get_rec_attrs(kwd_file, rec) -> dict:
    rec_attrs = dict()
    # names of metadata groups and locations in the rec group:
    fields_locs = {'rec_group': '/recordings/{}'.format(rec),
        'app_data': '/recordings/{}/application_data'.format(rec), 
        'cont_data': '/recordings/{}/data'.format(rec)}
    
    for name, location in fields_locs.items():
        rec_attrs[name] = h5tools.obj_attrs_2_dict_translator(kwd_file[location])
    return rec_attrs

get_rec_attrs(kwd_file, 0)

{'rec_group': {'bit_depth': 16,
  'sample_rate': 20000.0,
  'name': 0,
  'start_sample': 0,
  'start_time': 1},
 'app_data': {'is_multiSampleRate_data': 0,
  'channels_sample_rate': array([20000., 20000., 20000., 20000., 20000., 20000., 20000., 20000.,
         20000., 20000., 20000., 20000., 20000., 20000.]),
  'channel_bit_volts': array([1.9500e-01, 1.9500e-01, 1.9500e-01, 1.9500e-01, 1.9500e-01,
         1.9500e-01, 1.9500e-01, 1.9500e-01, 1.9500e-01, 1.9500e-01,
         1.9500e-01, 1.9500e-01, 1.9500e-01, 5.0354e-05]),
  'channel_names': array([b'D-004', b'D-006', b'D-008', b'D-012', b'D-019', b'D-022',
         b'D-023', b'D-024', b'D-025', b'D-026', b'D-027', b'D-028',
         b'D-029', b'ADC-00'], dtype='|S6')},
 'cont_data': {'valid_samples': array([14556240., 14556240., 14556240., 14556240., 14556240., 14556240.,
         14556240., 14556240., 14556240., 14556240., 14556240., 14556240.,
         14556240., 14556240.])}}

In [141]:
all_the_meta = 

SyntaxError: invalid syntax (<ipython-input-141-5965f6fbe54d>, line 1)

## Scripts for sorting

### prep the files with their nice formats, locations and names


In [142]:
rhd_list

NameError: name 'rhd_list' is not defined

In [143]:
ss_folder = os.path.join(exp_base, sess_par['bird'], 'ss_data', sess_par['sess'])

file_paths, out_folder = msc.make_paths(ss_folder)
os.makedirs(ss_folder, exist_ok=True)
#todo:
# meta and pars from intan files et al
s_f = 20000

# parameters to pass to the msort scripts
ds_params = {'samplerate': s_f,
            'detect_sign': -1}

with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))
    
# create the mda file from the rhd_list
#intan_meta, samples_in = intan.intan_to_bin(rhd_list[:], file_paths['mda'], variant='mda');


2018-12-18 18:33:33,063 - root - INFO - Created session par files /home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/params.json


In [144]:
ds_params

{'samplerate': 20000, 'detect_sign': -1}

In [145]:
file_paths

{'mda': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/raw.mda',
 'params': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/params.json',
 'geom': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/geom.csv',
 'filt': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/filt.mda.prv',
 'pre': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/pre.mda.prv',
 'firings': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings.mda',
 'firings_curated': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/firings_curated.mda',
 'cluster_metrics': '/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/cluster_metrics.json'}

In [146]:
# test mountainsort method for reading sorting parameters
msc.read_dataset_params(ss_folder)

{'samplerate': 20000, 'detect_sign': -1}

In [147]:
# Initialize a pipeline
from mountainlab_pytools import mlproc as mlp
Pipeline=mlp.initPipeline();

JSProxyWidget(status='Not yet rendered')

In [148]:
msc.sort_dataset?

Signature: msc.sort_dataset(*, file_paths:dict, freq_min:int=300, freq_max:int=6000, adjacency_radius:int=1, detect_threshold:float=3, dispatch_method:str='run', opts:dict={})
Docstring: <no docstring>
File:      ~/repos/pipefinch/pipefinch/neural/sort/mountain/core.py
Type:      function


In [149]:
output_dir = os.path.join(ss_folder, 'sort_out');
# Pipeline was failing, need to debug this ('NameError: name 'widgets' is not defined')
# with Pipeline:
#     msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=3, dispatch_method='add')

msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=2, freq_min=600);

2018-12-18 18:33:33,152 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/raw.mda --parameters freq_max:6000 freq_min:600 samplerate:20000 --outputs timeseries_out:/home/zinch/data/o9j9/ss_data/20181125_r021_saveontrigger/sort_out/filt.mda.prv


KeyboardInterrupt: 

In [ ]:
file_paths

## Command for viewing:
 - open up terminal with the environment msort
 - go go the ss_data folder for the session
 - run the command: qt-mountainview --raw raw.mda --filt sort_out/filt.mda.prv --pre sort_out/pre.mda.prv --samplerate=20000 --firings sort_out/firings.mda --cluster_metrics sort_out/cluster_metrics.json